In [2]:

# Importing required libraries
!pip install transformers datasets torch scikit-learn

from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score
import torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
# Ensure we're using GPU
print(torch.cuda.is_available())  # Should return True if GPU is available

True


In [5]:

# Step 1:  Load dataset and reduce size for quick testing
dataset = load_dataset("imdb")
small_train = dataset["train"].shuffle(seed=42).select(range(500))  # 500 samples for training
small_test = dataset["test"].shuffle(seed=42).select(range(100))  # 100 samples for evaluation


In [6]:

# Step 2: Load Pre-trained BERT Tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the data
def tokenize_data(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=128)

train_data = small_train.map(tokenize_data, batched=True)
test_data = small_test.map(tokenize_data, batched=True)

# Prepare dataset for PyTorch
train_data = train_data.rename_column("label", "labels")
test_data = test_data.rename_column("label", "labels")
train_data.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
test_data.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [7]:

# Step 3: Load Pre-trained BERT Model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:

# Step 4: Define Metrics for Evaluation
def compute_metrics(pred):
    labels = pred.label_ids
    preds = torch.argmax(torch.tensor(pred.predictions), dim=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}


In [23]:
!pip uninstall wandb -y


Found existing installation: wandb 0.18.7
Uninstalling wandb-0.18.7:
  Successfully uninstalled wandb-0.18.7


In [24]:
training_args = TrainingArguments(
    output_dir="./results",        # Output directory for saving the model
    num_train_epochs=3,            # Number of epochs
    per_device_train_batch_size=8, # Batch size for training
    per_device_eval_batch_size=8,  # Batch size for evaluation
    logging_dir="./logs",          # Directory for logs
    report_to="none",              # Disables W&B logging
)







In [25]:

# Step 6: Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
import os
os.environ["WANDB_DISABLED"] = "true"

<ipython-input-25-fa20ed4ead89>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [26]:

# Step 7: Train the Model
trainer.train()


Step,Training Loss


TrainOutput(global_step=189, training_loss=0.3477919240477224, metrics={'train_runtime': 34.2274, 'train_samples_per_second': 43.824, 'train_steps_per_second': 5.522, 'total_flos': 98666645760000.0, 'train_loss': 0.3477919240477224, 'epoch': 3.0})

In [29]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
# Evaluate the model on the test dataset
eval_results = trainer.evaluate()

# Print evaluation results
print(f"Evaluation results: {eval_results}")



Evaluation results: {'eval_loss': 0.8924707174301147, 'eval_accuracy': 0.77, 'eval_f1': 0.7526881720430108, 'eval_precision': 0.7608695652173914, 'eval_recall': 0.7446808510638298, 'eval_runtime': 0.3925, 'eval_samples_per_second': 254.764, 'eval_steps_per_second': 33.119, 'epoch': 3.0}


In [44]:

# Step 9: Test on Custom Sentences
def predict_sentiment(text):
    # Check if GPU is available and use it, otherwise fallback to CPU
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Move model to the appropriate device (GPU or CPU)
    model.to(device)

    # Tokenize the input sentence and move the input tensors to the same device as the model
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=128).to(device)

    # Make the prediction
    outputs = model(**inputs)

    # Get the predicted class
    predictions = torch.argmax(outputs.logits, dim=1)

    # Return the sentiment label based on the prediction
    return "Positive" if predictions == 1 else "Negative"

# Test predictions on custom sentences
test_sentence = input("Enter what you feel: ")  # Accept a full sentence as input
print(f"Text: {test_sentence} --> Sentiment: {predict_sentiment(test_sentence)}")  # Predict sentiment for the entire sentence


Enter what you feel: WELL GO TO HAVE SOME BREVAGES
Text: WELL GO TO HAVE SOME BREVAGES --> Sentiment: Positive
